In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from tensorflow.keras.metrics import categorical_crossentropy
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D
from keras.layers import Activation, Dropout, BatchNormalization, Flatten, Dense, AvgPool2D,MaxPool2D
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import backend as K
import cv2
import glob

In [45]:
!apt-get install -y libsm6 libxext6 libxrender-dev
!pip install opencv-python

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsm6 is already the newest version (2:1.2.2-1).
libxext6 is already the newest version (2:1.3.3-1).
libxrender-dev is already the newest version (1:0.9.10-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [46]:
!pip install tensorflow==2.6.0
!pip install keras~=2.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [47]:
!pip install -U efficientnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [48]:
from pathlib import Path
train_path = Path("/content/drive/MyDrive/2022_AI활용 표현과 문제해결/TermProject/Term_Dataset/train")
train_path.glob(r'**/*.png')

valid_path = Path("/content/drive/MyDrive/2022_AI활용 표현과 문제해결/TermProject/Term_Dataset/val")
valid_path.glob(r'**/*.png')

<generator object Path.glob at 0x7f4e4dd92650>

In [49]:
train_filepath = list(train_path.glob(r'**/*.png'))
valid_filepath = list(valid_path.glob(r'**/*.png'))

print(train_filepath)

[PosixPath('/content/drive/MyDrive/2022_AI활용 표현과 문제해결/TermProject/Term_Dataset/train/img00000.png'), PosixPath('/content/drive/MyDrive/2022_AI활용 표현과 문제해결/TermProject/Term_Dataset/train/img00001.png'), PosixPath('/content/drive/MyDrive/2022_AI활용 표현과 문제해결/TermProject/Term_Dataset/train/img00002.png'), PosixPath('/content/drive/MyDrive/2022_AI활용 표현과 문제해결/TermProject/Term_Dataset/train/img00016.png'), PosixPath('/content/drive/MyDrive/2022_AI활용 표현과 문제해결/TermProject/Term_Dataset/train/img00014.png'), PosixPath('/content/drive/MyDrive/2022_AI활용 표현과 문제해결/TermProject/Term_Dataset/train/img00009.png'), PosixPath('/content/drive/MyDrive/2022_AI활용 표현과 문제해결/TermProject/Term_Dataset/train/img00018.png'), PosixPath('/content/drive/MyDrive/2022_AI활용 표현과 문제해결/TermProject/Term_Dataset/train/img00010.png'), PosixPath('/content/drive/MyDrive/2022_AI활용 표현과 문제해결/TermProject/Term_Dat

In [50]:
os.path.split(os.path.split(train_filepath[0])[1]) 
    
    

('', 'img00000.png')

In [51]:
len(list(map(lambda x: os.path.split(os.path.split(x)[0])[1],valid_filepath)))

133

In [52]:
len(list(map(lambda x: os.path.split(os.path.split(x)[0])[1],train_filepath)))

864

In [53]:
classes_train = []
classes_valid = []


In [54]:
csv_file = pd.read_csv('/content/drive/MyDrive/2022_AI활용 표현과 문제해결/TermProject/Term_Dataset/train_data.csv')

print(csv_file['class'])

0       bag
1       bag
2       bag
3       bag
4       bag
       ... 
859    shoe
860    shoe
861    shoe
862    shoe
863    shoe
Name: class, Length: 864, dtype: object


In [55]:
csv_file = pd.read_csv('/content/drive/MyDrive/2022_AI활용 표현과 문제해결/TermProject/Term_Dataset/train_data.csv')

for i in range(1000):
  cnt = 0
  for j in csv_file.file_name:
    cnt = cnt + 1
    num = int(j[3:8])

    if(i == num):
      cnt2 = 0
      for k in csv_file['class']:
        cnt2 = cnt2 + 1
        if(cnt2 == cnt):
          classes_train.append(k)
          cnt = 0
          cnt2 = 0
          break
      

  

In [56]:
csv_file = pd.read_csv('/content/drive/MyDrive/2022_AI활용 표현과 문제해결/TermProject/Term_Dataset/val_data.csv')

for i in range(1000):
  cnt = 0
  for j in csv_file.file_name:
    cnt = cnt + 1
    num = int(j[3:8])

    if(i == num):
      cnt2 = 0
      for k in csv_file['class']:
        cnt2 = cnt2 + 1
        if(cnt2 == cnt):
          classes_valid.append(k)
          cnt = 0
          cnt2 = 0
          break
      


In [57]:
print(classes_train)

['bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed'

In [58]:
print(classes_train[72])

bed


In [59]:
print(len(classes_train))

864


In [60]:
print(classes_valid)

['bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bag', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'chair', 'chair', 'chair', 'chair', 'chair', 'chair', 'chair', 'chair', 'chair', 'chair', 'chair', 'coffeetable', 'coffeetable', 'coffeetable', 'coffeetable', 'coffeetable', 'coffeetable', 'coffeetable', 'coffeetable', 'coffeetable', 'coffeetable', 'coffeetable', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'kitchentools', 'kitchentools', 'kitchentools', 'kitchentools', 'kitchentools', 'kitchentools', 'kitchentools', 'kitchentools', 'kitchentools', 'kitchentools', 'kitchentools', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'lamp', 'laptop', 'laptop', 'laptop', 'laptop', 'laptop', 'laptop', 'laptop', 'laptop', 'laptop', 'LivingSofa', 'LivingSofa', 'LivingSofa', 'LivingSofa', 'LivingSofa', 'LivingSofa', 'L

In [61]:
print(len(classes_valid))

133


In [62]:
train_df = pd.concat([pd.Series(train_filepath).astype(str), pd.Series(classes_train)], axis=1)
train_df.columns = ['Images', 'Image_label']

valid_df = pd.concat([pd.Series(valid_filepath).astype(str), pd.Series(classes_valid)], axis=1)
valid_df.columns = ['Images', 'Image_label']


In [63]:
print(train_df)

                                                Images Image_label
0    /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...         bag
1    /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...         bag
2    /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...         bag
3    /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...         bag
4    /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...         bag
..                                                 ...         ...
859  /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...        shoe
860  /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...        shoe
861  /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...        shoe
862  /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...        shoe
863  /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...        shoe

[864 rows x 2 columns]


In [64]:
print(valid_df)

                                                Images Image_label
0    /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...         bag
1    /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...         bag
2    /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...         bag
3    /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...         bag
4    /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...         bag
..                                                 ...         ...
128  /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...        shoe
129  /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...        shoe
130  /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...        shoe
131  /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...        shoe
132  /content/drive/MyDrive/2022_AI활용 표현과 ᄆ...        shoe

[133 rows x 2 columns]


In [65]:
train_df.shape

(864, 2)

In [66]:
valid_df.shape

(133, 2)

In [67]:
train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, 
                                   shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True, vertical_flip =True)
test_datagen = ImageDataGenerator(rescale = 1.0/255.) 

In [68]:
train_gen = train_datagen.flow_from_dataframe(dataframe = train_df,
                                              x_col = 'Images', y_col ='Image_label',
                                              target_size = (200,200), batch_size = 512, 
                                              class_mode = 'categorical', shuffle = True)

val_gen = train_datagen.flow_from_dataframe(valid_df, 
                                            target_size=(200,200), x_col = 'Images', y_col ='Image_label', 
                                            class_mode='categorical',
                                            batch_size= 512, shuffle=True)
# test 도 있음        

Found 864 validated image filenames belonging to 11 classes.
Found 133 validated image filenames belonging to 11 classes.


In [69]:
train_gen.class_indices

{'LivingSofa': 0,
 'bag': 1,
 'bed': 2,
 'chair': 3,
 'coffeetable': 4,
 'cup': 5,
 'kitchentools': 6,
 'lamp': 7,
 'laptop': 8,
 'pot': 9,
 'shoe': 10}

In [70]:
#!pip install q tensorflow==2.1
#!pip install q keras==2.3.1

In [71]:
import efficientnet.keras as efn

base_model =  efn.EfficientNetB0(input_shape = (224, 224, 3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False
    
base_model.summary()  

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-71-f2716209b34a>", line 5, in <module>
    weights = 'imagenet')
  File "/usr/local/lib/python3.7/dist-packages/efficientnet/__init__.py", line 44, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/efficientnet/model.py", line 477, in EfficientNetB0
    **kwargs
  File "/usr/local/lib/python3.7/dist-packages/efficientnet/model.py", line 438, in EfficientNet
    model.load_weights(weights_path)
  File "/usr/local/lib/python3.7/dist-packages/keras/engine/saving.py", line 492, in load_wrapper
  File "/usr/local/lib/python3.7/dist-packages/keras/engine/network.py", line 1230, in load_weights
  File "/usr/local/lib/python3.7/dist-packages/keras/engine/saving.py", line 1183, in load_weights_from_hdf5_group
AttributeError: 'str' object has

AttributeError: ignored

In [ ]:
def display_random_image(class_names, images, labels):
    """
        Display a random image from the images array and its correspond label from the labels array.
    """
    
    index = np.random.randint(images.shape[0])
    plt.figure()
    #plt.imshow(images[index])
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    #plt.title('Image #{} : '.format(index) + class_names[labels[index]])
    plt.show()

In [ ]:
display_random_image(class_names, train_images, train_labels)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (150, 150, 3)), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(6, activation=tf.nn.softmax)
])

In [ ]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_images, train_labels, batch_size=128, epochs=20, validation_split = 0.2)